### Task 2  SKETCHING TASK 

TO DO
1. Build code for computing a Count-Min sketch,
2. play with different heights and widths for the Count-Min sketch matrix. 
3. Compare it to the MIN_WISE sampling strategy. 
4. Is it more space-efficient/accurate? What about run-time? 
5. Use the theory to explain any differences you observe.

ALGORITHM: COUNT - MIN SKETCH

count_min_sketch (over_count, return_minimum value, support subtraction)
    1. Initialize the 2D d x w array of counters
    2. Pick "d" hash functions for the i-th item
    3. For every hash in the hash function 
        counter of h(item) = counter of h(item) + 1
    4. For any "item" 
        report the minimum value of the h(item) as the estimation of frequency
    
    
item is ip_address
h is hash function 

In [52]:
#import nexessary libraries

import pandas as pd
#from mmh3 import hash
import murmurhash
from bitarray import bitarray
import numpy as np
import math


ModuleNotFoundError: No module named 'mmh3'

In [37]:
#Read the dataset of CTU-13 malware 43
dataset = pd.read_csv('dataset/capture20110811.pcap.netflow.labeled', delim_whitespace=True, chunksize=1000)
first_chunk = dataset.get_chunk()

# preprocess: clean up columns
def preprocess(chunk):
    chunk = chunk.rename(index=str, columns={
        "#Date_flow": "Date",
        "start": "Start",
        "Durat": "Duration",
        "Prot":"Protocol",
        "Src_IP_Addr:Port" : "Source_IP",
        "Dst_IP_Addr:Port": "Destination_IP",
        "Label(LEGITIMATE:Botnet:Background)": "Label"})
    chunk = chunk.drop(axis="columns", labels="->")
    return chunk

first_chunk = preprocess(first_chunk)
first_chunk.head()

,Date,Start,Duration,Protocol,Source_IP,Destination_IP,Flags,Tos,Packets,Bytes,Flows,Label
0,2011-08-11,10:10:00.003,0.967,UDP,89.31.8.11:23929,147.32.84.229:13363,INT,0,2,135,1,Background
1,2011-08-11,10:10:00.003,0.967,UDP,147.32.84.229:13363,89.31.8.11:23929,INT,0,2,276,1,Background
2,2011-08-11,10:10:00.006,0.000,UDP,208.88.186.6:34042,147.32.84.229:13363,INT,0,1,62,1,Background
3,2011-08-11,10:10:00.008,0.000,UDP,92.118.218.77:55246,147.32.84.229:13363,INT,0,1,78,1,Background
4,2011-08-11,10:10:00.009,0.000,UDP,182.185.139.181:10223,147.32.84.229:13363,INT,0,1,72,1,Background


In [35]:
len(first_chunk['Source_IP'].unique()) #555

555

In [36]:
len(first_chunk['Source_IP'])#1000
#first_chunk['Source_IP'] = '147.32.84.229';

1000

#### may be use this to find a particular ip in a list
aList = [123, 'xyz', 'zara', 'abc'];
print "Index for xyz : ", aList.index( 'xyz' ) 
print "Index for zara : ", aList.index( 'zara' ) 

In [25]:
d =  [1,23,44];
W = [2,23,65];

#count_min_matrix = np.array(d,W);
a = np.array([d,W]);
print(a);
print(a.dtype);
print(a[0][1]);

[[ 1 23 44]
 [ 2 23 65]]
int32
23


In [50]:
#1. Initialize the 2D d x w array of counters

d = 3; #rows - hash
W = 4; #columns - ip
CMS = [0] * d
for i in range(d):
    CMS[i] = [0] * W
#print(CMS)
# create a hash function to fill CMS with values
hash
CMS[0][1] =1;
print(CMS);

[[0, 1, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]


In [51]:
d = 3; #rows - hash
W = 4;
#2. Pick "d" hash functions for the i-th ipaddress
for i in range(d):
    for j in range(W):
        if (CMS[i][j] == 1):
            print(CMS[i][j]);
            print("i or d is ",i);
            print("j or W is ",j);
            
    
                


1
i or d is  0
j or W is  1
